**S03P03_tutorial_stateful_computations_in_jax.ipynb**

Arz

2024 APR 22 (MON)

reference:
https://jax.readthedocs.io/en/latest/jax-101/07-state.html

In [1]:
import numpy as np

In [2]:
import jax
import jax.numpy as jnp
from jax import lax
from jax import grad, jit, vmap
from jax import random

In [3]:
%xmode minimal

Exception reporting mode: Minimal


# motivation